In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
np.random.seed(42)

In [8]:
from Menu_v2 import Menu_of_classic_genetic_algorithm as Menu

In [9]:
nn_optimization = {"task": "nn_optimization",
                   "options": [{"data": None, "type": "dataset"}, 
                               {"target_name": "target", "type": "str"}, 
                               {"quality_type": 4, "type": "natural"},
                               {"outbreed_freq": 7, "type": "natural"},
                               {"n_epoch_mlp": 40, "type": "natural"},
                               {"eps_mlp": 20, "type": "natural"},
                               {"eta_mlp": 0.5, "type": "real"},
                               {"threshold": 0.2, "type": "threshold"},
                               {"elitism": 0.1, "type": "threshold"}]}

In [10]:
menu = Menu(nn_optimization)

Введите 'n_iter':
10
----------------------

Введите 'eps':

0.0001
----------------------

Выберите 'fitness_all_type'
(hint: нужно ввести номер варианта):
    1. average
    2. maximum
    3. minimum

2

2. maximum
----------------------

Выберите 'parent_selection_type'
(hint: нужно ввести номер варианта):
    1. roulette_wheel
    2. inbriding_phenotype
    3. inbriding_genotype
    4. panmixy
    5. choose_parent_nn

5

5. choose_parent_nn
----------------------

Выберите 'cross_type'
(hint: нужно ввести номер варианта):
    1. one_point
    2. discret_recombination_nn

2

2. discret_recombination_nn
----------------------

Введите 'p_cross':
1
----------------------

Выберите 'mutation_type'
(hint: нужно ввести номер варианта):
    1. binary
    2. mutation_nn

2

2. mutation_nn
----------------------

Введите 'p_mutation':
0.5
----------------------

Выберите 'select_new_population_type'
(hint: нужно ввести номер варианта):
    1. elite
    2. trunc
    3. exclusion
    4. selec

In [11]:
from  importlib import import_module

import feature_selection 
from choose_parent_types_file import *
from cross_types_file import *
from mutation_types_file import *
from select_new_population_file import *
from fitness_all_types_file import *

d_task = {"task":import_module(menu.task_with_options["task"])}

d_functions = {"parent_selection_type":import_module("choose_parent_types_file").__dict__[menu.options["parent_selection_type"]],
               "cross_type":import_module("cross_types_file").__dict__[menu.options["cross_type"]],
               "fitness_all_type":import_module("fitness_all_types_file").__dict__[menu.options["fitness_all_type"]],
               "mutation_type":import_module("mutation_types_file").__dict__[menu.options["mutation_type"]],
               "select_new_population_type":import_module("select_new_population_file").__dict__[menu.options["select_new_population_type"]]}

### Класс генетического алгоритма (специально не выносил в отдельный файл, чтобы был на виду)

In [12]:
import copy
from datetime import datetime

class GA:
    def __init__(self, d, d_task, d_functions):
        """
        Инициализация экземпляра
        Получает на вход словарь параметров алгоритма
        и распаковывает с созданием
        соответствующих атрибутов
        """
        self.d = d
        self.d_task = d_task
        self.d_functions = d_functions
        
        for key,value in d.items():
            setattr(self,key,value)
        for key,value in d_task.items():
            setattr(self,key,value)
        for key,value in d_functions.items():
            setattr(self,key,value)
        
            
    def generate(self):
        """
        Функция генерации популяции
        Дергает функцию генерации, которую делали прикладники
        Ничего не возвращает, но создает атрибут популяции, который
        сохраняется и далее используется
        """
        self.population = self.d_task["task"].generate(self.d)
    
    def fitness_one (self, osob):
        """
        Функция приспособленности особи
        Получает на вход ОСОБЬ
        Дергает функцию прикладников
        Возвращает приспособленность особи
        """
        return self.d_task["task"].fitness_one(self.d, osob)
    
    def validation(self,osob):
        """
        Функция проверки особи на допустимость
        Получает на вход особь
        Дергает функцию прикладников
        возвращает True/False
        """
        return self.d_task["task"].validation(self.d, osob)
    
    def fitness_all(self):
        """
        Функция приспособленности популяции
        Дергает функцию, которую я и делал
        Возвращает приспособленность популяции
        """
        return self.d_functions["fitness_all_type"](self.fitnesses)
        
        
    def choose_parent(self):
        """
        Функция выбора соседей.
        Если у нас n особей в популяции,
        то создается n//2 пар родителей,
        которые ПОТЕНЦИАЛЬНО могут скреститься
        Дергает функцию выбора родителей, которая
        в свою очередь возвращает список из двух номеров 
        особей, которые станут родителями
        Возвращает список из списков по два элемента с НОМЕРАМИ родителей в популяции
        (НЕ самими родителями)
        """
        lst_of_parents = []
        
        if self.d['parent_selection_type']=='choose_parent_nn':
            for i in range(len(self.population)//2):
                lst_of_parents.append(d_functions["parent_selection_type"](self.population, 
                                                                           self.fitnesses, 
                                                                           i,
                                                                           self.d))
        else:       
            for i in range(len(self.population)//2):
                lst_of_parents.append(self.d_functions["parent_selection_type"](self.population, self.fitnesses))
                
        return lst_of_parents
            
    def cross(self, parent1, parent2):
        """
        Функция скрещивания
        Принимает двух родителей и с соотв. вероятностью
        скрещивает их
        Дергает функцию скрещивания
        Возвращает двух особей-детей или список из 2 None,
        если скрещивание не удалось
        """
        c = random.random()
        if c < self.p_cross:
            return self.d_functions["cross_type"](parent1,parent2)
        else:
            return [None,None]
        
    def mutation(self,osob):
        """
        Функция, проводящая мутацию с конкретной особью с
        соответствующей вероятностью
        Получает на вход особь
        Дергает функцию мутации
        Возвращает мутированную особь
        """
        c = random.random()
        if c < self.p_mutation:
            return self.d_functions["mutation_type"](self.d, osob)
        else:
            return osob
                
    def select_new_population(self):
        """
        Функция, отбирающая из смешанной популяции
        родителей и детей size_of_population особей
        Дергает функцию выбора в новую популяцию
        Возвращает новую популяцию
        """
        return self.d_functions["select_new_population_type"](self.population, self.fitnesses, self.d)
            

    def fit(self):
        """
        Главный метод всего алгоритма, который 
        непосредственно проводит ГА и использует
        все предшествующие функции.
        ПЕчатает приспособленности на каждой итерации,
        возвращает 5 самых приспособленных особей
        """
        cnt = 0
        self.generate()
        self.population = list(filter(lambda osob: self.validation(osob), self.population)) # на всякий случай
        prev_fitness_all = float("-Inf")
        self.fitnesses = [self.fitness_one(osob) for osob in self.population]
        new_fitness_all = self.fitness_all()
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print (f"{now} После {cnt} итераций приспособленность популяции равна {new_fitness_all}")
        
        while (cnt < self.n_iter) or (new_fitness_all - prev_fitness_all > self.eps):
            
            prev_fitness_all = new_fitness_all
            prev_fitnesses = copy.deepcopy(self.fitnesses)
            prev_population = copy.deepcopy(self.population)
            cnt += 1
            
            lst_of_parents = self.choose_parent()
          
            lst_of_children = []
            for pair in lst_of_parents:
                child1, child2 = (self.cross(self.population[pair[0]], self.population[pair[1]]))
                if child1 is not None and self.validation(child1):
                    self.population.append(child1)
                    self.fitnesses.append(self.fitness_one(child1))
                if child2 is not None and self.validation(child2):
                    self.population.append(child2)
                    self.fitnesses.append(self.fitness_one(child2))
                    
            
            tmp_population = copy.deepcopy(self.population)
            
            for i,osob in enumerate(self.population):
                osob = self.mutation(osob)
                if self.validation(osob):
                    if osob != tmp_population[i]:
                        self.fitnesses[i] = self.fitness_one(osob)
                else:
                    self.fitnesses[i] = None
          
            self.population= list(filter(lambda osob: self.validation(osob), self.population))
            self.fitnesses = list(filter(lambda fitness: fitness is not None, self.fitnesses))
          
            self.population, self.fitnesses = self.select_new_population()
            new_fitness_all = self.fitness_all()
            now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            print (f"{now} После {cnt} итераций приспособленность популяции равна {new_fitness_all}")
            
            if new_fitness_all - prev_fitness_all < 0:
                self.population = prev_population
                self.fitnesses  = prev_fitnesses
                new_fitness_all = prev_fitness_all

        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')    
        print (f"{now}: Окончательная общая приспособленность", new_fitness_all)
        print ('-'*70)
        print ("Таблица результатов")
        for i in range(len(self.population)):
            print (self.population[i], self.fitnesses[i])
            # print (self.fitness_one(self.population[i]), self.fitnesses[i])
        return self.population[:5]

### Инициализация экземпляра класса

In [13]:
ga = GA(menu.options, d_task, d_functions)

### Проведение эволюционного алгоритма

In [14]:
%%time
ga.fit()

2019-10-15 00:01:46 После 0 итераций приспособленность популяции равна 0.85
2019-10-15 00:02:28 После 1 итераций приспособленность популяции равна 0.8395061728395061
2019-10-15 00:03:06 После 2 итераций приспособленность популяции равна 0.8500000000000001
2019-10-15 00:03:40 После 3 итераций приспособленность популяции равна 0.8363636363636363
2019-10-15 00:04:19 После 4 итераций приспособленность популяции равна 0.8500000000000001
2019-10-15 00:04:56 После 5 итераций приспособленность популяции равна 0.8500000000000001
2019-10-15 00:05:35 После 6 итераций приспособленность популяции равна 0.8571428571428572
2019-10-15 00:06:16 После 7 итераций приспособленность популяции равна 0.8780487804878048
2019-10-15 00:06:54 После 8 итераций приспособленность популяции равна 0.9268292682926829
2019-10-15 00:07:28 После 9 итераций приспособленность популяции равна 0.9268292682926829
2019-10-15 00:08:06 После 10 итераций приспособленность популяции равна 0.9375
2019-10-15 00:08:40 После 11 итерац

[[0.10110000000000004, 261, 'tanh', 'tanh'],
 [0.10110000000000004, 261, 'tanh', 'tanh'],
 [0.11920000000000056, 236, 'tanh', 'sigm'],
 [0.11920000000000056, 268, 'tanh', 'sigm'],
 [0.11920000000000056, 268, 'tanh', 'sigm']]